In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout
from keras.callbacks import EarlyStopping

2024-08-27 07:13:13.263048: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-27 07:13:13.265426: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-27 07:13:13.272578: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 07:13:13.285249: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 07:13:13.288849: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-27 07:13:13.298314: I tensorflow/core/platform/cpu_feature_gu

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
# Loading dataset
df = pd.read_csv('../clean_data/sarcasm_headlines_v2_clean.csv')

In [4]:
# Checking if dataset is correctly loaded
df.head()

,is_sarcastic,cleaned_headline
0,1,thirtysomething scientist unveil doomsday cloc...
1,0,dem rep totally nail why congress be fall shor...
2,0,eat your veggie deliciously different recipe
3,1,inclement weather prevent liar from get to work
4,1,mother come pretty close to use word stream co...


In [5]:
df.shape

(28619, 2)

In [6]:
# Checking max length
len(df['cleaned_headline'].max())

92

In [7]:
# Model parms
max_len = 100
embedding_dim = 300
max_features = 5000
num_filters = 32

# Model fit params
batch_size = 128
num_epochs = 8

In [8]:
# Define X and y
X = df['cleaned_headline']
y = df['is_sarcastic']

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenizing
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index

# Converting words to integers
sequences = tokenizer.texts_to_sequences(X)

# Padding the sequences
sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

In [10]:
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(sequences, y, test_size=.3, random_state=42, shuffle=True)

In [11]:
# Checking shape
X_train.shape

(20033, 100)

In [12]:
words = []
for sentence in df['cleaned_headline'].values:
    words.append(sentence.split())

In [18]:
# Importing word2vec model
import gensim

# Load pre-trained Word2Vec model.
w2v_model = gensim.models.Word2Vec(sentences=words, window = 5, min_count = 1)

# Create an embedding matrix
embedding_dim = 300  # Word2Vec Google News vectors have 300 dimensions
embedding_matrix = np.zeros((max_features, embedding_dim))

for word, i in word_index.items():
    if i < max_features:
        embedding_vector = word2vec.get_vector(word) if word in w2v_model else None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

TypeError: argument of type 'Word2Vec' is not iterable

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional


model = Sequential([
    Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len),  # Embedding layer
    Bidirectional(LSTM(64, return_sequences=True)),  # Bidirectional LSTM
    Dropout(0.5),  # Dropout layer to prevent overfitting
    LSTM(32),  # Another LSTM layer
    Dropout(0.5),
    Dense(32, activation='relu'),  # Fully connected layer
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Defining early stopping params
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    verbose = 1,
    restore_best_weights = True
)

In [ ]:
# Train the model
history = model2.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[es])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

In [ ]:
# Training the model
model.fit(X_train, y_train, batch_size=32, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[es])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')